### Load PDF Document from Google Drive

In [2]:
!pip install gdown -q 
!pip install groq  -q
!pip install qdrant-client -q
!pip install sentence-transformers -q
!pip install langchain -q
!pip install pypdf  -q 

In [3]:
import os
import gdown
from groq import Groq
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv

/home/game/ai_innovator_llm_rag/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
load_dotenv('.env')  
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
PDF_ID = os.getenv("PDF_ID")

In [5]:
# Google Drive PDF file ID
pdf_id = PDF_ID  

# Destination path for the downloaded PDF
pdf_path = "./dataset/downloaded_document.pdf"  

# Download the PDF file from Google Drive
gdown.download(f"https://drive.google.com/uc?id={pdf_id}", pdf_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1Tlf5jxYlvScKUAhNkUxYDkWrcXp1gEIM
To: /home/game/ai_innovator_llm_rag/notebook/dataset/downloaded_document.pdf
100%|██████████| 88.2k/88.2k [00:00<00:00, 1.08MB/s]


'./dataset/downloaded_document.pdf'

In [6]:
# Initialize embedding model using Sentence Transformers
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_size = embedding_model.embed_query("test").__len__()

/tmp/ipykernel_51062/1551404959.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [7]:
# Initialize Qdrant Client (using in-memory or adjust to your needs)
qdrant = QdrantClient(":memory:")

# Create a Qdrant collection to hold document vectors
qdrant.recreate_collection(
    collection_name="documents",
    vectors_config={"size": vector_size, "distance": "Cosine"}
)

/tmp/ipykernel_51062/4230452278.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [8]:
# Load PDF document
loader = PyPDFLoader(pdf_path) 
# Load the downloaded PDF
documents = loader.load()

In [9]:
# Split the documents into smaller chunks
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document_chunks = splitter.split_documents(documents)

# Extract text from document chunks
texts = [doc.page_content for doc in document_chunks]

# Convert texts into vectors
vectors = embedding_model.embed_documents(texts)

# Upsert data into Qdrant
points = [PointStruct(id=i, vector=vectors[i], payload={"text": texts[i]}) for i in range(len(texts))]
qdrant.upsert(collection_name="documents", points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [10]:
def search_documents(query):
    # Convert query to vector
    query_vector = embedding_model.embed_query(query)
    
    # Search Qdrant for similar documents
    search_results = qdrant.search(
        collection_name="documents",
        query_vector=query_vector,
        limit=3  # Retrieve top 3 relevant documents
    )
    
    # Check if results are found
    if not search_results:
        return []  # Return empty if no documents found

    # Extract text from results
    return [hit.payload.get("text", "เอกสารไม่มีข้อความ") for hit in search_results]

In [11]:
def generate_answer(query):
    # Retrieve relevant documents from Qdrant
    retrieved_docs = search_documents(query)
    
    # Check if any documents were retrieved
    if not retrieved_docs:
        return "ไม่พบข้อมูลที่เกี่ยวข้อง"  # "No relevant information found"

    # Create the context for the language model
    context = "\n".join([str(doc) for doc in retrieved_docs if isinstance(doc, str)])
    
    if not context.strip():  # Check if context has content
        return "ไม่พบข้อมูลที่เกี่ยวข้อง"  # "No relevant information found"

    prompt = f"ข้อมูลอ้างอิง:\n{context}\n\nคำถาม: {query}\n\nคำตอบ:"
    
    # Initialize Groq API client
    groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))

    try:
        # Requesting completion from Groq API
        response = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"เกิดข้อผิดพลาดในการสร้างคำตอบ: {str(e)}"  # "Error occurred in generating the answer"

In [12]:
# 🔥 Test the question!
query = "มีรายการอะไรบ้าง"  # "What are the contents?"
answer = generate_answer(query)
print(answer)

/tmp/ipykernel_51062/1751635943.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant.search(


มีรายการที่ 25 รายการ ในใบแจ้งหนังสือยาที่คุณให้มา


In [13]:
# 🔥 Test the question!
query = "ขอรายอะเอียดของยา  พาราเซตามอล(Paracetamol)"  
answer = generate_answer(query)
print(answer)

/tmp/ipykernel_51062/1751635943.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant.search(


ยาภาราเซตามอล (Paracetamol) เป็นยาที่ใช้ในการลดการแพงที่เกิดจากภาวะไข้, อาการปวด, และอาการอ่อนเพลีย มีลักษณะของยาในรูปแบบของเม็ด, สารรักษาโรคประเภทโอปิอิดของต้นไม้และของพรรณไม้ที่ใช้ในการบำบัดปวดท้องและอาการปวดที่มีอาการปวดของกล้ามเนื้อ

วิธีการใช้พาราเซตามอล:

* สำหรับยาที่ออกตามสูตรผสม: ใช้พาราเซตามอล 500-1000 มิลลิกรัม ทุกๆ 4-6 ชั่วโมง ไม่เกิน 4 ครั้งต่อวัน
* สำหรับเด็กอายุ 4-5 ปี: ใช้พาราเซตามอล 250-500 มิลลิกรัม ทุกๆ 4-6 ชั่วโมง ไม่เกิน 4 ครั้งต่อวัน
* สำหรับเด็กอายุต่ำกว่า 4 ปี: ห้ามใช้พาราเซตามอลได้

ตัวอย่าง: เมื่อคุณมีอาการปวดหรือไข้ คุณสามารถใช้พาราเซตามอลตามคำแนะนำของผู้เชี่ยวชาญหรือตามคำแนะนำบนฉลากของยาได้

ระวัง: ห้ามใช้พาราเซตามอลไปแล้ว 4 ชั่วโมงก่อนที่จะต่อสู้กับเครื่องบิน คำแนะนำ: เมื่อคุณใช้พาราเซตามอล ควรใช้น้อยลงเมื่อคุณมีขนาดตัวเล็กขึ้น
